In [1]:
# !pip install neptune > /dev/null 2>&1
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import neptune

In [13]:
def get_full_table(tags, start_end, component_name, download=True, exp_rate=None, table=None, negative_tags=None, granularities=None):
    if download:
        tags.append(start_end)
        tags.append(component_name)
        project = neptune.init_project(
            project="pmtest/llm-random",
            mode="read-only",
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyMDY0ZDI5Ni05YWU3LTQyNGYtYmY4My1hZTFkY2EzYmUwMjgifQ=="
        )
        # columns = ["loss", "args/name", "args/learning_rate", "args/relative_lr/embedding_layer", "args/relative_scheduler_fraction/projection", "args/relative_lr/gating", "args/relative_lr/expert_inner_function", "args/relative_lr/head", "args/final_lr_fraction", "args/relative_scheduler_fraction/embedding_layer", "args/relative_lr/projection", "args/relative_scheduler_fraction/gating", "args/relative_scheduler_fraction/expert_inner_function", "args/relative_scheduler_fraction/head", "args/dmodel"]
        start_end_dict_name = "relative_lr"
        if start_end != "start":
            start_end_dict_name = "relative_scheduler_fraction"
        arg_val_column = f"{start_end_dict_name}/{component_name}"
        print(arg_val_column)
        columns = ["loss", f"args/{arg_val_column}"]
        table = project.fetch_runs_table(tag=tags).to_pandas()
        table = table[table["sys/state"] == "Inactive"]
        table = table[table["args/n_steps"] == table["step"]]
        # table = table[table["args/learning_rate"] == 2e-4]
        if exp_rate != None:
            table = table[table["args/expansion_rate"] == exp_rate]

        # table["active_params"] = table.apply(lambda row: calculate_active_params(dmodel=row["args/dmodel"], n_blocks=row["args/n_blocks"]), axis=1)
        print(f"Number of experiments: {len(table)}")
    print("All sizes and token counts in table:")
    return table[columns]

In [14]:
start_end = "start"
component_name = "head"
tags = ["relative_lr", "medium", "local_minimum"]
table = get_full_table(tags=tags, start_end=start_end, component_name=component_name)
print(table)

https://app.neptune.ai/pmtest/llm-random/
relative_lr/head
Number of experiments: 5
All sizes and token counts in table:
       loss  args/relative_lr/head
0  3.686363                   1.00
1  3.726680                   2.66
2  3.734096                   1.33
3  3.649711                   0.33
4  3.645006                   0.17
